## Import libraries

In [ ]:
from urllib.parse import urlparse
from arcgis.mapping import WebMap
from arcgis.gis import GIS

## Connect to _source_ and _target_ GIS organizations

In [ ]:
source = GIS(profile="your_ent_admin_profile", verify_cert=False)
target = GIS(profile="cloning_admin", verify_cert=False)

In [ ]:
print(source)

In [ ]:
print(target)

## Search for specific `Web Mapping Application`

In [ ]:
wmas = source.content.search(query="Elephants *", 
                            item_type="Web Mapping Application",
                            max_items=100)
wmas

### Get the `Web Mapping Application` from the resulting list

In [ ]:
habitat_story = wmas[0]
habitat_story

## Get the `Web Mapping Application` data 

 * return the list of `keys` available from the resulting definition

In [ ]:
wma_def = habitat_story.get_data()
wma_def

In [ ]:
list(wma_def.keys())

### Get the _source_ item used in the application

In [ ]:
source_item = source.content.get(wma_def['source'])
source_item

Now we know this is a `Story Map` application.

### Get the `values` for the application

In [ ]:
wma_def['values']

### Get the `Webmap` item used in this `Story Map`

In [ ]:
storymap_webmap_id = wma_def['values']['webmap']
storymap_webmap = source.content.get(storymap_webmap_id)
storymap_webmap

##### Helper Function

We wrote a function, taking a `Webmap` item and `GIS` object as inputs, to quickly tell us whether a `Webmap` has already been cloned in a particular system represented by our Python API `GIS` object.

In [ ]:
def is_webmap_cloned(src_webmap, target_gis):
    target_webmaps = target_gis.content.search(query="* AND NOT owner:esri*",
                                               item_type="Web Map",
                                               max_items=500)
    for tgt_webmap in target_webmaps:
        for kw in tgt_webmap.typeKeywords:
            if src_webmap.id in kw:
                return tgt_webmap
                break
    return None

### Determine if `Webmap` has already been cloned

In [ ]:
cloned_wm = is_webmap_cloned(storymap_webmap, target)

if cloned_wm:
    print(f"{storymap_webmap.title} {storymap_webmap.id} cloned in {target.url}\n"
          f"...{cloned_wm.title}:{cloned_wm.id}")
else:
    print(f"{storymap_webmap.title} {storymap_webmap.id} NOT cloned in {target.url}")

### Print information about any layers in the `Webmap`

In [ ]:
orig_wm_obj = WebMap(storymap_webmap)
for orig_wm_lyr in orig_wm_obj.layers:
    print(f"{orig_wm_lyr['title']}")
    print(f"{orig_wm_lyr['url']}")
    try:
        if source.content.get(orig_wm_lyr['itemId']):
            print(f"{orig_wm_lyr['itemId']}")
    except Exception as e:
        pass
    print("\n")

### Search `Webmap` for any layers that have previously been cloned

In [ ]:
for layer in orig_wm_obj.layers:
    try:
        source_item = source.content.get(layer['itemId'])
        for flyr_item in target.content.search(query="*", 
                                               item_type="Feature Layer", 
                                               max_items=2000):
            for kw in flyr_item.typeKeywords:
                if source_item.id in kw:
                    print(f"{layer['title']}\n{'-'*80}")
                    print(f"...{source_item.title} {source_item.type} cloned in {target.url}")
                    print(f"{' '*5}---Source: {source_item.id}{' '*4}Target: {flyr_item.id}")
                    print(f"\n")
                    break
    except Exception as e:
        pass

## Clone the `Webmap` used in the `Web Mapping Application`

 * create `wm_mapping` dictionary with _source_ `Webmap.id` as key and _target_ cloned `Webmap.id` as value to subsequently use when cloning the `Web Mapping Application item`

In [ ]:
cloned_from_storymap = target.content.clone_items(items=[storymap_webmap],
                                                 folder="Elephant_Habitat")
wm_mapping = {storymap_webmap.id:cloned_from_storymap[0].id}
cloned_from_storymap

### Get the cloned `Webmap` from the resulting list

In [ ]:
cloned_storymap_wm = cloned_from_storymap[0]
cloned_storymap_wm

### Print information about the layers in the cloned `Webmap`
 * compare the beginning of the path compenent in the `url` to the organization id value retrieved in the previous step
 * if we can get an `item` from the `itemId` value, print out the `itemId` value for the new `item` created in the _target_

In [ ]:
cloned_wm_obj = WebMap(cloned_storymap_wm)
for cloned_wm_lyr in cloned_wm_obj.layers:
    print(f"{cloned_wm_lyr['title']}")
    print(f"{cloned_wm_lyr['url']}")
    try:
        if target.content.get(cloned_wm_lyr['itemId']):
            print(f"{cloned_wm_lyr['itemId']}")
    except Exception as e:
        pass
    print("\n")

In [ ]:
cloned_storymap_wm.id

### Verify the `wm_mapping` parameter contains `itemId` values

In [ ]:
wm_mapping

## Clone the _source_ `Web Mapping Application`

 * the `wm_mapping` parameter instructs the function to use the _target_ `Webmap` item instead of the _source_ `Webmap`

In [ ]:
cloned_wma = target.content.clone_items(items=[habitat_story],
                                        folder="Elephant_Habitat",
                                        item_mapping=wm_mapping)
cloned_wma

### Get the `Story Map` from the resulting list

In [ ]:
cloned_storymap = cloned_wma[0]
cloned_storymap

### Compare the `Webmap` id value from the _source_ `Story Map` to the _target_ 

In [ ]:
cloned_sm_def = cloned_storymap.get_data()
cloned_sm_def['values']['webmap']

In [ ]:
target.content.get(cloned_sm_def['values']['webmap'])